# Anime Recommender System

---

### Essential Libraries

Let us begin by importing the essential Python Libraries.

> NumPy : Library for Numeric Computations in Python  
> Pandas : Library for Data Acquisition and Preparation  
> Matplotlib : Low-level library for Data Visualization  
> Seaborn : Higher-level library for Data Visualization  

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

---

## Collect data from myanimelist.net API

Test with one anime first to see what features are available

In [2]:
# library for fetching api
import requests

In [3]:
# query for anime
response_query = requests.get("https://api.myanimelist.net/v2/anime?q=Shigatsu&limit=4", 
                        headers={'X-MAL-CLIENT-ID': '6114d00ca681b7701d1e15fe11a4987e'})
print("Response:", response_query.status_code)

Response: 200


In [4]:
response_query.json()

{'data': [{'node': {'id': 23273,
    'title': 'Shigatsu wa Kimi no Uso',
    'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/3/67177.jpg',
     'large': 'https://api-cdn.myanimelist.net/images/anime/3/67177l.jpg'}}},
  {'node': {'id': 28069,
    'title': 'Shigatsu wa Kimi no Uso: Moments',
    'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/6/74156.jpg',
     'large': 'https://api-cdn.myanimelist.net/images/anime/6/74156l.jpg'}}},
  {'node': {'id': 38091,
    'title': 'Hachigatsu no Cinderella Nine (TV)',
    'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/1824/100449.jpg',
     'large': 'https://api-cdn.myanimelist.net/images/anime/1824/100449l.jpg'}}},
  {'node': {'id': 5597,
    'title': 'Natsu no Arashi!',
    'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/5/75256.jpg',
     'large': 'https://api-cdn.myanimelist.net/images/anime/5/75256l.jpg'}}}],
 'paging': {'next': 'https://api.my

In [5]:
# extract anime id and title, did use the anime picture
anime = response_query.json()['data'][0]['node']
anime_id = anime['id']
anime_title = anime['title']
anime

{'id': 23273,
 'title': 'Shigatsu wa Kimi no Uso',
 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/3/67177.jpg',
  'large': 'https://api-cdn.myanimelist.net/images/anime/3/67177l.jpg'}}

In [6]:
# anime details
response_details = requests.get(f'https://api.myanimelist.net/v2/anime/{anime_id}?fields=id,title,main_picture,alternative_titles,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,my_list_status,num_episodes,start_season,broadcast,source,average_episode_duration,rating,pictures,background,related_anime,related_manga,recommendations,studios,statistics',
                        headers={'X-MAL-CLIENT-ID': '6114d00ca681b7701d1e15fe11a4987e'})
print("Response:", response_details.status_code)

Response: 200


In [7]:
print("Features Avaiable:")
print("-----")
for feature in response_details.json().keys():
    print(feature)

Features Avaiable:
-----
id
title
main_picture
alternative_titles
start_date
end_date
synopsis
mean
rank
popularity
num_list_users
num_scoring_users
nsfw
created_at
updated_at
media_type
status
genres
num_episodes
start_season
broadcast
source
average_episode_duration
rating
pictures
background
related_anime
related_manga
recommendations
studios
statistics


---
- **From the features list above, we want to `keep` the following features:**
  - id
  - title
  - start_date
  - end_date
  - synopsis
  - mean
  - rank
  - popularity
  - num_list_users
  - num_scoring_users
  - nsfw
  - created_at
  - updated_at
  - media_type
  - status
  - genres
  - num_episodes
  - start_season
  - broadcast
  - source
  - average_episode_duration
  - rating
  - pictures
  - background
  - studios
  - statistics
- **We have `dropped` the following features:**
  - main_picture
  - alternative_titles
  - pictures
  - related_anime
  - related_manga
  - recommendations

*Future response_details queries will only include the listed features above*

---

### Create dataframe to store data

In [8]:
anime_data_list = []

### Get top ranked animes in various categories

In [9]:
ranking_categories = [
    'all',
    'airing',
    'tv',
    'ova',
    'movie',
    'special',
    'bypopularity',
    'favorite'
]

ranking_type = ranking_categories[0]
response_ranking = requests.get(f'https://api.myanimelist.net/v2/anime/ranking?ranking_type={ranking_type}&limit=100',
                               headers={'X-MAL-CLIENT-ID': '6114d00ca681b7701d1e15fe11a4987e'})
print('Status', response_ranking.status_code)

Status 200


In [10]:
'''
Get the anime ids
response_ranking.json()['data'][0]['node']['id']
'''

for anime in response_ranking.json()['data']:
    anime_id = anime['node']['id']
    
    # query for anime details
    response_details = requests.get(f'https://api.myanimelist.net/v2/anime/{anime_id}?fields=id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,my_list_status,num_episodes,start_season,broadcast,source,average_episode_duration,rating,background,studios,statistics',
                        headers={'X-MAL-CLIENT-ID': '6114d00ca681b7701d1e15fe11a4987e'})
    
    anime_data_list.append(response_details.json())
    


In [11]:
df = pd.DataFrame(anime_data_list)
df

,id,title,main_picture,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,...,genres,num_episodes,start_season,broadcast,source,average_episode_duration,rating,background,studios,statistics
0,5114,Fullmetal Alchemist: Brotherhood,{'medium': 'https://api-cdn.myanimelist.net/im...,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.15,1,3,2812347,...,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",64,"{'year': 2009, 'season': 'spring'}","{'day_of_the_week': 'sunday', 'start_time': '1...",manga,1460,r,,"[{'id': 4, 'name': 'Bones'}]","{'status': {'watching': '215153', 'completed':..."
1,28977,Gintama°,{'medium': 'https://api-cdn.myanimelist.net/im...,2015-04-08,2016-03-30,"Gintoki, Shinpachi, and Kagura return as the f...",9.09,2,337,514467,...,"[{'id': 1, 'name': 'Action'}, {'id': 4, 'name'...",51,"{'year': 2015, 'season': 'spring'}","{'day_of_the_week': 'wednesday', 'start_time':...",manga,1440,pg_13,This is a fourth season of Gintama. In the ep...,"[{'id': 1258, 'name': 'Bandai Namco Pictures'}]","{'status': {'watching': '57682', 'completed': ..."
2,9253,Steins;Gate,{'medium': 'https://api-cdn.myanimelist.net/im...,2011-04-06,2011-09-14,The self-proclaimed mad scientist Rintarou Oka...,9.09,3,13,2184663,...,"[{'id': 8, 'name': 'Drama'}, {'id': 40, 'name'...",24,"{'year': 2011, 'season': 'spring'}","{'day_of_the_week': 'wednesday', 'start_time':...",visual_novel,1460,pg_13,Steins;Gate is based on 5pb. and Nitroplus' vi...,"[{'id': 314, 'name': 'White Fox'}]","{'status': {'watching': '141470', 'completed':..."
3,38524,Shingeki no Kyojin Season 3 Part 2,{'medium': 'https://api-cdn.myanimelist.net/im...,2019-04-29,2019-07-01,Seeking to restore humanity's diminishing hope...,9.08,4,31,1749834,...,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",10,"{'year': 2019, 'season': 'spring'}","{'day_of_the_week': 'monday', 'start_time': '0...",manga,1435,r,Shingeki no Kyojin adapts content from volumes...,"[{'id': 858, 'name': 'Wit Studio'}]","{'status': {'watching': '67475', 'completed': ..."
4,48583,Shingeki no Kyojin: The Final Season Part 2,{'medium': 'https://api-cdn.myanimelist.net/im...,2022-01-10,2022-04-04,Turning against his former allies and enemies ...,9.08,5,190,736112,...,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",12,"{'year': 2022, 'season': 'winter'}","{'day_of_the_week': 'monday', 'start_time': '0...",manga,1435,r,,"[{'id': 569, 'name': 'MAPPA'}]","{'status': {'watching': '474642', 'completed':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,40776,Haikyuu!! To the Top 2nd Season,{'medium': 'https://api-cdn.myanimelist.net/im...,2020-10-03,2020-12-19,"Once called a fallen powerhouse and known as ""...",8.54,96,252,620484,...,"[{'id': 4, 'name': 'Comedy'}, {'id': 8, 'name'...",12,"{'year': 2020, 'season': 'fall'}","{'day_of_the_week': 'saturday', 'start_time': ...",manga,1436,pg_13,,"[{'id': 10, 'name': 'Production I.G'}]","{'status': {'watching': '55179', 'completed': ..."
96,48661,JoJo no Kimyou na Bouken Part 6: Stone Ocean,{'medium': 'https://api-cdn.myanimelist.net/im...,2021-12-01,NaN,"In Florida, 2011, Jolyne Kuujou sits in a jail...",8.54,97,606,304805,...,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",0,"{'year': 2021, 'season': 'fall'}",NaN,manga,1469,r,,"[{'id': 287, 'name': 'David Production'}]","{'status': {'watching': '166857', 'completed':..."
97,4282,Kara no Kyoukai 5: Mujun Rasen,{'medium': 'https://api-cdn.myanimelist.net/im...,2008-08-16,2008-08-16,"In November 1998, a double homicide occurs at ...",8.54,98,886,214965,...,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",1,"{'year': 2008, 'season': 'summer'}",NaN,light_novel,6894,r,Includes claymation short which was shown befo...,"[{'id': 43, 'name': 'ufotable'}]","{'status': {'watching': '4037', 'completed': '..."
98,35843,Gintama.: Porori-hen,{'medium': 'https://api-cdn.myanimelist.net/im...,2017-10-02,2017-12-25,"Following the grim events of Iga, Kok